In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import davies_bouldin_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load the datasets
customers_df = pd.read_csv('Customers.csv')
transactions_df = pd.read_csv('Transactions.csv')

# Merge datasets on CustomerID
df = pd.merge(customers_df, transactions_df, on='CustomerID')

# Handle missing values (if any)
df = df.dropna()

# Feature engineering: create new features based on transactions
df['TotalSpent'] = df.groupby('CustomerID')['Price'].transform('sum')
df['TransactionFrequency'] = df.groupby('CustomerID')['ProductID'].transform('count')

# Normalize data for clustering
features = ['TotalSpent', 'TransactionFrequency', 'Region', 'SignupDate']  # Example features
X = df[features]

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Try different numbers of clusters and calculate DB Index
db_index = []
k_range = range(2, 11)  # Trying from 2 to 10 clusters
for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    
    # Calculate Davies-Bouldin Index
    db_index.append(davies_bouldin_score(X_scaled, kmeans.labels_))

# Find the optimal number of clusters based on DB Index (lowest is best)
optimal_k = k_range[np.argmin(db_index)]
print(f"Optimal number of clusters: {optimal_k}")

# Final clustering with optimal clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_scaled)

# Calculate the final DB Index
final_db_index = davies_bouldin_score(X_scaled, df['Cluster'])
print(f"Final Davies-Bouldin Index: {final_db_index}")

# Apply PCA for 2D visualization
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Plot the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(x=X_pca[:, 0], y=X_pca[:, 1], hue=df['Cluster'], palette="viridis", s=100, alpha=0.7)
plt.title(f'Customer Segmentation (K={optimal_k})')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend(title='Cluster')
plt.show()
